# Адаптивный алгоритм интегрирования с помощью формул Ньютона-Котеса.
# Интеграл с особыми точками.


In [3]:
import math

### Задача - посчитать интеграл $\int_0^1 \frac{\ln(1 - x^2)}{x}dx$. 

Для этого разложим его в сумму трёх интегралов: первый интеграл ограничен в нуле, но подынтегральная функция неопределена в этой точке, поэтому доопределим подынтегральную функцию по непрерывности в точке $0$. У второго интеграла нет особых точек. В последнем интеграле сделаем отступ от точки $1$ так, чтобы он посчитался с нужной погрешностью.

$$\int_0^1 \frac{\ln(1 - x^2)}{x}dx = \int_0^{\frac12} \frac{\ln(1 - x^2)}{x}dx + \int_{\frac12}^1 \frac{\ln(1 + x)}{x}dx + \int_{\frac12}^1 \frac{\ln(1 - x)}{x}dx$$

Промажорируем остаток интеграла в окрестности особой точки. 

Положим $\varepsilon > 0$ - некоторое достаточно малое число. Воспользуемся тем, что любая степенная функция растёт быстрее, чем логарифм. Начиная с некоторого $\epsilon$ будет выполнено неравенство:

$$ \left|\int_{1 - \varepsilon}^1 \frac{\ln(1 - x)}{x}dx \right| \le \int_{1 - \varepsilon}^1 \frac{dx}{(1 - \varepsilon)\sqrt{1 - x}} = 
\frac1{1 - \varepsilon} \int_0^{\varepsilon} \frac{dt}{\sqrt{t}} = \frac1{2(1 - \varepsilon)}\sqrt{\varepsilon}  \le \frac{\delta}2$$

Допустим, нужно посчитать интеграл с некоторой погрешностью $\delta = 10^{-4}$.

Оценим хвост интеграла величиной $\delta * 0.5$.

Решая квадратное неравенство получаем $\sqrt{\varepsilon} \le \dfrac{-\frac1{\delta * 0.5} + \sqrt{\frac1{\delta^2 * 0.25} + 4}}{2}$

Следовательно, $\varepsilon$ можем взять следующим:

In [4]:
delta = 1e-4

eps = (-1/(delta * 0.5) + math.sqrt(1/delta**2 / 0.25 + 4)) ** 2


In [5]:
import numpy as np
import matplotlib.pyplot as plt
xx = np.linspace(0 + 1e-1, 1, 50)
plt.plot(xx, -np.log(xx))
g = lambda x: 1 / x**0.5
plt.plot(xx, g(xx))

In [21]:
#формула пярмоугольников
def Integrate(h, f, a, b):
    value = 0
    i = 0
    while(abs(a + h * i - b) > 1e-10):
        value += f(a + h * i + h / 2) 
        i += 1
    return value * h

#реализация алгоритма
def solve(a, b, f, epsA = 1e-4 / 2, epsO = 1e-4):
    alpha, beta = a, b
    I = E = 0
    while alpha < beta:        
        h = beta - alpha
        I_h = Integrate(h, f, alpha, beta)
        I_hdiv2 = Integrate(h / 2, f, alpha, beta)
                
        delta = (I_hdiv2 - I_h) / (2**3 - 1) # p = 3
        
        if abs(delta) < max(epsA, epsO * abs(I)) * h / (b - a):
            alpha = beta
            beta = min(beta + h, b)
            I = I + I_hdiv2 + delta
            E = E + delta
            
        if abs(delta) > max(epsA, epsO * abs(I)) * h / (b - a):
            beta = (alpha + beta) / 2

        elif abs(delta) < max(epsA, epsO * abs(I)) / 2 ** 3 * h / (b - a):
            beta = min(alpha + 2 * h, b)

    return I, E

In [22]:
ans = 0
f = lambda x: 0 if x == 0 else math.log(1 - x**2) / x
ans += solve(0, 0.5, f)[0]
f = lambda x: math.log(1 + x) / x
ans += solve(0.5, 1, f)[0]
f = lambda x: math.log(1 - x) / x
ans += solve(0.5, 1 - eps, f)[0]
ans

-0.8224283372759821

Как и ожидалось $S(f)\approx -\dfrac{\pi^2}{12}$.

In [23]:
-math.pi ** 2 / 12

-0.8224670334241132